### Import and create an empty TMG

In [1]:
# import igraph
import logging
import matplotlib.pyplot as plt 

from dredFISH.Analysis import TissueGraph
from dredFISH.Analysis import Classification
from dredFISH.Utils import tmgu

import importlib
importlib.reload(TissueGraph)
importlib.reload(Classification)
importlib.reload(tmgu)

logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')

In [26]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t4'
!ls -alhtr $basepth
!head $basepth"/TMG.json"

total 61M
drwxrwxrwx 7 zach     wollmanlab 4.0K Jul  8 12:44 ..
lrwxrwxrwx 1 fangming wollmanlab   69 Jul  8 12:45 DPNMF_PolyA_2021Nov19_Section_-1850X_270Y_metadata.csv -> ../Dataset1-t1/DPNMF_PolyA_2021Nov19_Section_-1850X_270Y_metadata.csv
lrwxrwxrwx 1 fangming wollmanlab   67 Jul  8 12:45 DPNMF_PolyA_2021Nov19_Section_-1850X_270Y_matrix.csv -> ../Dataset1-t1/DPNMF_PolyA_2021Nov19_Section_-1850X_270Y_matrix.csv
drwxr-xr-x 2 fangming wollmanlab 4.0K Jul  8 12:52 .
-rw-r--r-- 1 fangming wollmanlab  165 Jul 12  2022 TMG.json
-rw-r--r-- 1 fangming wollmanlab  54M Jul 12  2022 cell.h5ad
-rw-r--r-- 1 fangming wollmanlab 6.8M Jul 12  2022 isozone.h5ad
-rw-r--r-- 1 fangming wollmanlab 700K Jul 12  2022 region.h5ad
-rw-r--r-- 1 fangming wollmanlab    3 Jul 12  2022 Taxonomy_clusters.csv
-rw-r--r-- 1 fangming wollmanlab    3 Jul 12  2022 Taxonomy_topics.csv
{"layers_graph": [[0, 1], [0, 2]], "layer_taxonomy_mapping": {"0": 0, "1": 0, "2": 1}, "Taxonomies": ["clusters", "topics"], "Layers": ["

In [3]:
TMG = TissueGraph.TissueMultiGraph(basepath=basepth, 
                                   redo=True, # create an empty one
                                  ) 

### Create a `cell` layer
Creating a cell layer, load data from file, normalizes and creates an unclassified tissue graph

In [4]:
%%time
TMG.create_cell_layer(metric = 'cosine')
logging.info(f"TMG has {len(TMG.Layers)} Layers")

INFO:root:In TMG.create_cell_layer
INFO:root:Started reading matrices and metadata
INFO:root:done reading files
INFO:root:77846 cells, minimum counts = 1872.0
INFO:root:building spatial graphs
INFO:root:Building spatial graphs for 1 sections


/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/cell.h5ad


INFO:root:updating anndata
INFO:root:done building spatial graph
INFO:root:building feature graphs
INFO:root:building feature graph using cosine
INFO:root:done with create_cell_layer
INFO:root:TMG has 1 Layers


CPU times: user 1min 32s, sys: 1min 21s, total: 2min 53s
Wall time: 39.3 s


### Create Geometries

In [5]:
# %%time
# TMG.add_geoms()

CPU times: user 3 µs, sys: 3 µs, total: 6 µs
Wall time: 13.1 µs


### Create cell types using `OptLeiden classifier` applied on the `cell` layer

In [6]:
# %%time
# # Create the classifier
# optleiden = Classification.OptimalLeidenKNNClassifier(TMG.Layers[0])
# # train the classifier
# optleiden.train(opt_res=11.5, opt_params={'iters':10, 'n_consensus':1})
# # use the classifier to create types and add them to TMG using the Taxonomy created on the fly by the classifier
# type_vec = optleiden.classify()
# TMG.add_type_information(0, type_vec, optleiden.tax)

/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad
Number of types: 153 initial entropy: -6.770682785610586 number of evals: 0
CPU times: user 23 s, sys: 62 ms, total: 23.1 s
Wall time: 23 s


### Create `isozone` layer

In [7]:
# TMG.create_isozone_layer()
# logging.info(f"TMG has {len(TMG.Layers)} Layers")

INFO:root:TMG has 2 Layers


/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad


### Create `region` layer
To create regions, we first create a new classifier that works on local cell environments (local type abundace). Classify cells based on their regions and use this type to create a new layer. 

In [8]:
# %%time

# n_topics_list = [2,5,10]
# n_procs = 3 

# topic_cls = Classification.TopicClassifier(TMG.Layers[0])
# topic_cls.train(n_topics_list=n_topics_list, n_procs=n_procs)
# topics = topic_cls.classify(topic_cls.Env)

INFO:root:Running LDA in parallel with 3 cores


CPU times: user 24.3 s, sys: 3.08 s, total: 27.4 s
Wall time: 3min 15s


In [14]:
topics
import numpy as np

In [15]:
np.unique(topics, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([9209, 5397, 6471, 8152, 9229, 7502, 9462, 5830, 9130, 7464]))

In [21]:
topics.shape

(77846,)

In [20]:
topic_cls.tax.Type

[]

In [30]:
CG = TMG.Layers[0].contract_graph(topics)

/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3/isozone.h5ad


In [36]:
CG.Upstream.shape

(77846,)

In [35]:
np.unique(CG.Upstream, return_counts=True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]),
 array([3797, 1077, 3787, 8261, 2631, 9203, 2710, 8648, 1658,    1,   57,
         360,  382,  318,  429, 5729, 4543,    1,   41,   14, 3575,   27,
           1,  146,    1,    3, 1303,    6,   24, 6369, 5488,    6, 1334,
         174,  181,    3,  182,    1,    3,   11,    1,    6,    1,   82,
          59,    1,    1,    1,  443, 1519,    1, 

In [23]:
%%time
TMG.create_region_layer(topics, topic_cls.tax)
logging.info(f"TMG has {len(TMG.Layers)} Layers")

INFO:root:TMG has 3 Layers


!!`region` layer already exists; return...
CPU times: user 4.48 ms, sys: 246 µs, total: 4.73 ms
Wall time: 3.36 ms


In [27]:
TMG.Layers[2].Type.shape

(116,)

### Save to files
TMG is saved as a config json file, one AnnData file per layer, and one dataframe per taxonomy. 

In [25]:
%%time
TMG.save()

INFO:root:saved


CPU times: user 154 ms, sys: 119 ms, total: 273 ms
Wall time: 815 ms
